In [1]:
#import dependencies
import numpy as np
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func
from sqlalchemy.orm import Session

from flask import Flask, jsonify

#database setup
database_path = "../Resources/hawaii.sqlite"
engine = create_engine(f"sqlite:///{database_path}")

#reflect an existing database into a new model
Base = automap_base()
#reflect the tables
Base.prepare(autoload_with=engine)

In [2]:
#save database references to a table
Measurement = Base.classes.measurement
Station = Base.classes.station

In [3]:
#flask setup
app = Flask(__name__)

In [4]:
@app.route("/")
def welcome():
    """List all available api routes"""
    return(
        f"Available Routes:<br/>"
        f"/api/v1.0/precipitation<br/>"
        f"/api/v1.0/stations<br/>"
        f"/api/v1.0/tobs<br/>"
    )

In [5]:
#app routing for precip for the past 12 months
@app.route("/api/v1.0/precipitation")
def precipitation():
    #create session link
    session = Session(engine)
    #query the last 12 months of precip data
    cutoff_date = '2016-08-23'
    results = session.query(Measurement.date, Measurement.prcp).filter(Measurement.date > cutoff_date).all()
    session.close()

    #create a dictionary from the results and append to a list of precipitation_data
    precip_data = []
    for date, prcp in results:
        precip_dict = {}
        precip_dict["date"] = date
        precip_dict["prcp"] = prcp
        precip_data.append(precip_dict)

    return jsonify(precip_data)



In [6]:
 #app routing for station list
@app.route("/api/v1.0/stations")
def stations():
    #create session link
    session = Session(engine)
    #query the names of all stations in the list
    results = session.query(Measurement.station).distinct().all()
    session.close()

    #create a dictionary of the active stations and their counts
    station_data = []
    for station in results:
        station_dict = {}
        station_dict["station name"] = station
        station_data.append(station_dict)

    return jsonify(station_data)

In [7]:
#app routing for t_observed for the past 12 months
@app.route("/api/v1.0/tobs")
def tobs():
    #create session link
    session = Session(engine)
    #query the last 12 months of temperature data from the most active observation station 
    cutoff_date_12month = '2016-08-23'
    results = session.query(Measurement.tobs).filter((Measurement.station == 'USC00519281') & (Measurement.date > cutoff_date_12month)).all()
    session.close()

    #create a dictionary of t_obs data for the most active station
    tobs_data = []
    for tobs in results:
        tobs_dict = {}
        tobs_dict["Oberved Temperature"] = tobs
        tobs_data.append(tobs_dict)

    return jsonify(tobs_data)


In [8]:
#app routing for min, max, avg temp for a given start date
@app.route("/api/v1.0/")
def temps_start(start_date):
    session = Session(engine)

    results = session.query(func.avg(Measurement.tobs), func.min(Measurement.tobs), func.max(Measurement.tobs)).filter(Measurement.date >= start_date).all()

    temp_data = []
    for tobs in results:
        temp_dict = {}
        temp_dict["Average"] = func.avg(Measurement.tobs)
        temp_dict["Minimum"] = func.min(Measurement.tobs)
        temp_dict["Maximum"] = func.max(Measurement.tobs)
        temp_data.append(temp_dict)

    return jsonify(temp_data)
    

In [9]:
#app routing for min, max, avg temp for a given start date
@app.route("/api/v1.0/")
def temps_start_end(start_date, end_date):
    session = Session(engine)

    results = session.query(func.avg(Measurement.tobs), func.min(Measurement.tobs), func.max(Measurement.tobs)).\
    filter((Measurement.date >= start_date)&(Measurement.date <= end_date)).\
    all()

    temp_data = []
    for tobs in results:
        temp_dict = {}
        temp_dict["Average"] = func.avg(Measurement.tobs)
        temp_dict["Minimum"] = func.min(Measurement.tobs)
        temp_dict["Maximum"] = func.max(Measurement.tobs)
        temp_data.append(temp_dict)

    return jsonify(temp_data)

In [10]:
if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with watchdog (windowsapi)


SystemExit: 1

c:\Users\John\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
